In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env

env = make_vec_env(
    "HalfCheetah-v4",
    rng=np.random.default_rng(),
    n_envs=4,
    env_make_kwargs={"render_mode": "rgb_array"},
)


expert = load_policy(
    "sac-huggingface",
    organization="sb3",
    env_name="HalfCheetah-v3",
    venv=env,
)

C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: 'bytes' object cannot be interpreted as an integer
  warnings.warn(
C:\Users\xavier\anaconda3\envs\RL\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend transitioning to Gymnasium by saving that model again.
  warnings.warn(


In [2]:
from src.models.dagger import TrajectoryAccumulator
from src.models.dagger import generate_trajectories

In [3]:
obs = env.reset()
print(obs.shape)
print(env.num_envs,env.observation_space.shape[0] ,env.action_space.shape[0])

(4, 17)
4 17 6


In [4]:
trajectories_accum = TrajectoryAccumulator(env.num_envs,env.observation_space.shape[0] ,env.action_space.shape[0])

partial_trajectories {0: [], 1: [], 2: [], 3: []}


In [5]:
init_acts = np.zeros((env.num_envs, env.action_space.shape[0]))
init_rews = np.zeros(env.num_envs)
init_dones = np.zeros(env.num_envs, dtype=bool)

for env_idx, (act, rew, next_ob, done) in enumerate(zip(init_acts, init_rews, obs, init_dones)):
    step_dict = dict(acts=act, rews=rew, next_obs=next_ob, dones=done)
    trajectories_accum.add_step(env_idx, step_dict)

In [6]:
acts, _ = expert.predict(obs)
print("action", acts.shape)

action (4, 6)


In [7]:
next_obs, rews, dones, infos = env.step(acts)
print(next_obs.shape)
print(rews.shape)
print(dones.shape)
print(len(infos))

(4, 17)
(4,)
(4,)
4


In [8]:
t = trajectories_accum.get_partial_trajectories()

In [9]:
traj = trajectories_accum.add_step_and_auto_finish(acts,rews,next_obs,np.ones(env.num_envs, dtype=bool))

In [10]:
traj

[{'acts': array([0., 0., 0., 0., 0., 0.]),
  'rews': 0.0,
  'next_obs': array([-0.04604266, -0.0918053 , -0.09669447,  0.06265405,  0.08255112,
          0.02132716,  0.04589931,  0.008725  , -0.12654215, -0.06232745,
          0.0041326 , -0.23250308, -0.02187917, -0.12459109, -0.07322674,
         -0.0544259 , -0.03163002]),
  'dones': False},
 {'acts': array([-0.88531256,  0.83578444, -0.82933587, -0.4214357 , -0.6741191 ,
         -0.26134658], dtype=float32),
  'rews': -0.91246,
  'next_obs': array([-0.08519518, -0.09489906, -0.35278949,  0.36813254, -0.29212861,
         -0.09529661, -0.15064564, -0.02839609, -0.77367833, -1.09972993,
          0.08039848, -6.32742959,  6.39726559, -8.97898448, -3.71776041,
         -5.26592797, -1.2587513 ]),
  'dones': True}]

In [11]:
for env_idx, (act, rew, next_ob, done) in enumerate(zip(acts, rews, next_obs, dones)):
    step_dict = dict(acts=act, rews=rew, next_obs=next_ob, dones=done)
    trajectories_accum.add_step(env_idx, step_dict)

partial_trajectories {0: [{'acts': array([0., 0., 0., 0., 0., 0.]), 'rews': 0.0, 'next_obs': array([-0.04604266, -0.0918053 , -0.09669447,  0.06265405,  0.08255112,
        0.02132716,  0.04589931,  0.008725  , -0.12654215, -0.06232745,
        0.0041326 , -0.23250308, -0.02187917, -0.12459109, -0.07322674,
       -0.0544259 , -0.03163002]), 'dones': False}, {'acts': array([-0.88531256,  0.83578444, -0.82933587, -0.4214357 , -0.6741191 ,
       -0.26134658], dtype=float32), 'rews': -0.91246, 'next_obs': array([-0.08519518, -0.09489906, -0.35278949,  0.36813254, -0.29212861,
       -0.09529661, -0.15064564, -0.02839609, -0.77367833, -1.09972993,
        0.08039848, -6.32742959,  6.39726559, -8.97898448, -3.71776041,
       -5.26592797, -1.2587513 ]), 'dones': False}], 1: [{'acts': array([0., 0., 0., 0., 0., 0.]), 'rews': 0.0, 'next_obs': array([ 0.09009274, -0.07116808,  0.08972989, -0.03763371, -0.01533471,
        0.06554052, -0.01816017,  0.00991874,  0.02941325,  0.00284222,
       

In [ ]:
step_dict = dict(acts=act, rews=rew, next_obs=next_ob, dones=done)
trajectories_accum.add_step(env_idx, step_dict)